In [1]:
import numpy as np

import pycram
import pycram.orm.object_designator
import sqlalchemy

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [98]:
from pycram.bullet_world import BulletWorld, Object

world = BulletWorld()
kitchen = Object("kitchen", "environment", "kitchen.urdf")

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 2060/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 530.30.02
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 530.30.02
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 2060/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
dest

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Scalar element defined multiple times: limit
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']


In [99]:
import pycram.bullet_world_reasoning as btr

milk = Object("Milk", "milk", "milk.stl", position=[1, 0, 1])
pr2 = Object("pr2", "robot", "pr2.urdf")

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [154]:
kitchen.links

{'room_link': 0,
 'sink_area_footprint': 1,
 'sink_area': 2,
 'sink_area_surface': 3,
 'sink_area_sink': 4,
 'sink_area_right_panel': 5,
 'sink_area_trash_drawer_main': 6,
 'sink_area_trash_drawer_handle': 7,
 'sink_area_left_upper_drawer_main': 8,
 'sink_area_left_upper_drawer_handle': 9,
 'sink_area_left_middle_drawer_main': 10,
 'sink_area_left_middle_drawer_handle': 11,
 'sink_area_left_bottom_drawer_main': 12,
 'sink_area_left_bottom_drawer_handle': 13,
 'sink_area_dish_washer_main': 14,
 'sink_area_dish_washer_door': 15,
 'sink_area_dish_washer_door_handle': 16,
 'oven_area_area_footprint': 17,
 'oven_area_area': 18,
 'oven_area_oven_main': 19,
 'oven_area_oven_door': 20,
 'oven_area_oven_door_handle': 21,
 'oven_area_oven_panel': 22,
 'oven_area_oven_knob_stove_1': 23,
 'oven_area_oven_knob_stove_2': 24,
 'oven_area_oven_knob_stove_3': 25,
 'oven_area_oven_knob_stove_4': 26,
 'oven_area_oven_knob_oven': 27,
 'oven_area_area_middle_upper_drawer_main': 28,
 'oven_area_area_middle_

In [128]:
"""
Dieser Part funktioniert noch nicht, da die Costmap immer dieselbe Pose sampled.
"""

from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
import time
for x in range(1000):
    time.sleep(1.5)
    milk.set_position_and_orientation([0,2,1],[0,0,0,0.66])
    milk_desig = BelieveObject(names=["Milk"]).resolve()
    kitchen_description = ObjectDesignatorDescription(names=['kitchen']).resolve()
    object_locations = SemanticCostmapLocation(urdf_link_name="kitchen_island_surface", part_of=kitchen_description,
                                               for_object=milk_desig).resolve()
    milk.set_position_and_orientation(object_locations.pose[0], object_locations.pose[1])
    print(object_locations.pose[0])
    robot_desig = BelieveObject(types=["robot"]).resolve()
    location_description = CostmapLocation(target=milk_desig, reachable_for=robot_desig)
    location_description.resolve()
    world.reset_bullet_world()


for pose in location_description:
    p = pose.pose
    pr2.set_position_and_orientation(p[0],p[1])
    is_visible = CostmapLocation(target=milk_desig, visible_for=robot_desig)
    print(is_visible.resolve())

AttributeError: 'Object' object has no attribute 'bullet_world_object'

In [ ]:
"""
Benutze stattdessen den pose_generator um Posen für die Milchpackung zu generieren
"""

from pycram.enums import Arms
from pycram.designators.action_designator import ParkArmsAction
import pycram.costmaps as cm
from pycram.process_module import simulated_robot, with_simulated_robot

s = cm.SemanticCostmap(kitchen, "kitchen_island_surface", size=100, resolution=0.02)
milk_aabb = milk.get_AABB()
# Sample 1000 poses on the kitchen island surface
poses = pycram.pose_generator_and_validator.pose_generator(costmap=s, number_of_samples=1000)
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

#Todo: Fix collision and clipping through the walls.
for pose in poses:
    print(pose)
    # Without this offset, the milkbottle clips into the islands surface
    offset = np.sqrt(np.square(pose[0][2] - milk_aabb[0][2]))
    position = np.array([pose[0][0],pose[0][1],pose[0][2] + offset])
    milk.set_position(position)
    robot_desig = BelieveObject(types=["robot"]).resolve()
    location_description = CostmapLocation(target=milk_desig, reachable_for=robot_desig)
    for pose in location_description:
        p = pose.pose
        pr2.set_position_and_orientation(p[0],p[1])
        # Is object visible for the pr2
        is_visible = CostmapLocation(target=milk_desig, visible_for=robot_desig)
        break

([-1.1675000190734863, 2.259200096130371, 0.8410000205039978], [-0.0, 0.0, 0.6394961783590891, -0.7687942753845921])
([-1.1675000190734863, 2.2192001342773438, 0.8410000205039978], [-0.0, 0.0, 0.6339889056055383, -0.7733421413379021])
([-1.1675000190734863, 2.2992000579833984, 0.8410000205039978], [-0.0, 0.0, 0.6442410481961081, -0.7648225100107736])
([-1.1675000190734863, 2.199199914932251, 0.8410000205039978], [-0.0, 0.0, 0.6308905049634007, -0.7758718777910598])
([-1.1675000190734863, 2.339200019836426, 0.8410000205039978], [-0.0, 0.0, 0.6483700815767094, -0.7613253163505146])
([-1.1675000190734863, 2.2392001152038574, 0.8410000205039978], [-0.0, 0.0, 0.6368487253657931, -0.7709887813710162])
([-1.1675000190734863, 2.379199981689453, 0.8410000205039978], [-0.0, 0.0, 0.6519948536035278, -0.7582233911417626])
([-1.1675000190734863, 2.4191999435424805, 0.8410000205039978], [-0.0, 0.0, 0.6552017233487013, -0.7554539706169343])
([-1.1675000190734863, 2.1791999340057373, 0.841000020503997

In [126]:
world.reset_bullet_world()

In [97]:
world.exit()